# Modules for Datajoint

In [3]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/notebooks/meshAfterParty/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

Connecting celiib@at-database.ad.bcm.edu:3306


In [4]:
import minfig
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


In [5]:
minnie = minfig.configure_minnie(return_virtual_module=True)

# Old way of getting access to the virtual modules
# m65 = dj.create_virtual_module('minnie', 'microns_minnie65_02')

#New way of getting access to module
import datajoint as dj
from minfig import adapter_objects # included with wildcard imports
minnie = dj.create_virtual_module('minnie', 'microns_minnie65_02', add_objects=adapter_objects)

schema = dj.schema("microns_minnie65_02")
dj.config["enable_python_native_blobs"] = True
#(schema.jobs & "table_name='__baylor_segment_centroid_seg3'").delete()

INFO:datajoint.settings:Setting enable_python_native_blobs to True
INFO:datajoint.settings:Setting enable_python_native_blobs to True


In [7]:
np.exp(15)

3269017.3724721107

In [19]:
mesh_n_vertices = minnie.Mesh.fetch("n_vertices")

In [20]:
np.min(mesh_n_vertices),np.max(mesh_n_vertices)

(460, 74011386)

In [11]:
meshes_decimated = (minnie.Mesh() & minnie.Decimation.proj())

In [14]:
n_vertices_decimated = meshes_decimated.fetch("n_vertices")

In [21]:
np.min(n_vertices_decimated),np.max(n_vertices_decimated)

(162757, 3268979)

In [18]:
np.round(np.exp(15))

3269017.0

In [8]:
decimation_version = 0
decimation_ratio = 0.25

segments = (minnie.Mesh() & f'n_vertices  > {np.str(np.round(np.exp(12)).astype(np.int))}' & f'n_vertices  < {np.str(np.round(np.exp(15)).astype(np.int))}')
key_source =  minnie.Decimation.proj(decimation_version='version') & segments.proj() & "decimation_version=" + str(decimation_version)
key_source  

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691134649778727,0,0.10
864691134697247994,0,0.10
864691134718953212,0,0.10
864691134721124348,0,0.10
864691134734371578,0,0.10
864691134743838460,0,0.10
864691134746633978,0,0.10
864691134748184152,0,0.10
864691134759873880,0,0.10
864691134760657252,0,0.10


In [36]:
mesh_with_nucleus_dec = minnie.Decimation() & minnie.NucleusID.proj() & "decimation_ratio=0.25"
mesh_with_nucleus_dec

segment_id segment id unique within each Segmentation,version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),n_vertices,n_faces,mesh in-place path to the hdf5 (decimated) mesh file
864691134829367042,0,0.25,72131,144983,=BLOB=
864691134831878402,0,0.25,141431,288006,=BLOB=
864691134832796418,0,0.25,64192,129768,=BLOB=
864691134836870914,0,0.25,103573,208897,=BLOB=
864691134868397058,0,0.25,125239,252340,=BLOB=
864691134873997050,0,0.25,48159,97321,=BLOB=
864691134875433210,0,0.25,66493,133498,=BLOB=
864691134875454714,0,0.25,138619,278598,=BLOB=
864691134875960570,0,0.25,57675,115610,=BLOB=
864691134876055298,0,0.25,86259,176911,=BLOB=


In [49]:
mesh_with_nucleus & f"n_vertices < {np.exp(12)}" & f"n_vertices > {np.exp(11)}"

segment_id segment id unique within each Segmentation,n_vertices number of vertices,n_faces number of faces,mesh in-place path to the hdf5 mesh file
864691135898530280,97789,190926,=BLOB=
864691135896080616,123207,245791,=BLOB=
864691135905885097,87256,170311,=BLOB=
864691135563379140,74119,147664,=BLOB=
864691136107882466,92500,181624,=BLOB=
864691135653759123,72017,143127,=BLOB=
864691136110548049,120133,237098,=BLOB=
864691135629790597,61205,121142,=BLOB=
864691135536515177,107905,214943,=BLOB=
864691136143273367,146508,291889,=BLOB=


In [48]:
np.exp(11)

59874.14171519782

In [56]:
small_soma_mesh = fetch_undecimated_segment_id_mesh(seg_id = 864691135898530280)

In [57]:
import neuron_visualizations as nviz
nviz.plot_objects(meshes=small_soma_mesh)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [23]:
mesh_with_nucleus = minnie.Mesh() & minnie.NucleusID.proj()

segment_id segment id unique within each Segmentation,n_vertices number of vertices,n_faces number of faces,mesh in-place path to the hdf5 mesh file
864691135087399350,10385,20774,=BLOB=
864691136303487127,6853,13456,=BLOB=
864691135113166262,10386,20760,=BLOB=
864691135511469390,41167,82830,=BLOB=
864691135931295367,28805,57681,=BLOB=
864691135936438151,28535,57047,=BLOB=
864691136227222513,23921,48212,=BLOB=
864691134663756538,33754,67846,=BLOB=
864691135948558983,12919,25926,=BLOB=
864691135841854896,23914,48064,=BLOB=


In [39]:
np.exp(10)

22026.465794806718

In [26]:
n_vertices_nuclei = mesh_with_nucleus.fetch("n_vertices")
np.min(n_vertices_nuclei),np.max(n_vertices_nuclei)

(4660, 32377896)

In [34]:
bins[1]

3241983.6

In [24]:
valid_segment_ids_with_nucleus_id = dj.U("segment_id") & (minnie.NucleusID() & "segment_id>0")
valid_segment_ids_with_nucleus_id

segment_id segment_id merged to nucleus
864691136050815731
864691135084884330
864691136521572241
864691135494586958
864691136451074559
864691135524436073
864691135087399350
864691134684964653
864691135524409705
864691134917390346


# The tables that should be used for pulling meshes

In [4]:
decimation_ratio = 0.25
decimation_version = 0

valid_segment_ids_with_nucleus_id = dj.U("segment_id") & (minnie.NucleusID() & "segment_id>0")
segments = (minnie.Mesh())# & f'n_vertices  > {np.str(np.round(np.exp(12)).astype(np.int))}' & f'n_vertices  < {np.str(np.round(np.exp(15)).astype(np.int))}')
key_source =  (minnie.Decimation.proj(decimation_version='version') 
            & segments.proj() 
            & f"decimation_ratio={decimation_ratio}" 
            & f"decimation_version={decimation_version}" 
            & valid_segment_ids_with_nucleus_id)
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691135635239593,0,0.25
864691135618175635,0,0.25
864691135584809548,0,0.25
864691135679304325,0,0.25
864691136143741847,0,0.25
864691136148842647,0,0.25
864691136143284631,0,0.25
864691136745551332,0,0.25
864691136315868311,0,0.25
864691136324529047,0,0.25


In [42]:
def nucleus_id_to_seg_id(nucleus_id):
    """
    Pseudocode:
    1) restrict the nucleus id table
    2) fetch for the segment id
    """
    nucl_key = dict(nucleus_id=nucleus_id)
    nucl_seg_id = (minnie.NucleusID()  & nucl_key).fetch1("segment_id")
    if nucl_seg_id == 0:
        raise Exception(f"No segment id for nucleus Id {nucleus_id}")
    return nucl_seg_id
    
import trimesh
def fetch_segment_id_mesh(seg_id,decimation_ratio=0.25):
    key = dict(segment_id=seg_id,decimation_ratio=decimation_ratio)
    new_mesh = (minnie.Decimation() & key).fetch1("mesh")
    current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces
    return trimesh.Trimesh(vertices=current_mesh_verts,faces=current_mesh_faces)

def fetch_undecimated_segment_id_mesh(seg_id,decimation_ratio=0.25):
    key = dict(segment_id=seg_id,decimation_ratio=decimation_ratio)
    new_mesh = (minnie.Mesh() & key).fetch1("mesh")
    current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces
    return trimesh.Trimesh(vertices=current_mesh_verts,faces=current_mesh_faces)


def fetch_nucleus_id_mesh(nucleus_id,verbose=False):
    seg_id = nucleus_id_to_seg_id(nucleus_id)
    print(f"Attempting to fetch segment_id: {seg_id}")
    return fetch_segment_id_mesh(seg_id)

In [9]:
n_verts,seg_ids = (segments & key_source & "n_vertices > 3000000").fetch("n_vertices","segment_id")

In [10]:
np.max(n_verts)

3268979

In [8]:
from os import sys
sys.path.append("/meshAfterParty/")
import neuron_visualizations as nviz

# Viewing the Neurons

In [16]:
curr_seg_id = seg_ids[7]
print(curr_seg_id)

864691135548568516


In [17]:
new_mesh = fetch_segment_id_mesh(curr_seg_id)
new_mesh

<trimesh.Trimesh(vertices.shape=(764384, 3), faces.shape=(1523580, 3))>

In [18]:
nviz.plot_objects(meshes=new_mesh)

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [19]:
curr_seg_id

864691135548568516

In [20]:
# import trimesh_utils as tu
# tu.write_neuron_off(new_mesh,"864691135548568516_single_soma_inhib_axon_cloud.off")

'864691135548568516_single_soma_inhib_axon_cloud.off'

In [ ]:
single_soma_excitatory_large_axon = [864691135419631858]
single_soma_excitatory_large_axon_with_merges = [864691135577756100]
single_soma_inhibitory_large_axon = [864691136265012708]
double_soma_large_axon = [864691135382617330]
triple_soma_with_lots_axon = [864691136819525998]
single_soma_inhib_axon_cloud = [864691135548568516]
lots_of_error = [864691135707063188] #probably has a blood vessel